In [1]:
import pandas as pd

In [2]:
!pip install gensim==4.3.2 numpy==1.23.5 pandas==1.5.3

In [3]:
!pip install scipy==1.12

In [4]:
import gensim
import numpy
import scipy

In [5]:
print(gensim.__version__)
print(numpy.__version__)
print(scipy.__version__)

4.3.2
1.23.5
1.12.0


In [ ]:
# Charger le fichier et vérifier les colonnes.

In [6]:
df = pd.read_csv("glassdoor_reviews_proceeded.csv")
df.head()

,Unnamed: 0,firm,date_review,overall_rating,work_life_balance,culture_values,career_opp,comp_benefits,senior_mgmt,recommend,...,headline_clean,pros_clean,cons_clean,full_text_clean,proceeded_pros_clean,final_pros_clean,proceeded_cons_clean,final_cons_clean,proceeded_headline,final_headline
0,0,Apple,2008-01-31,5,3.0,5.0,3.0,4.0,4.0,v,...,product poeple enjoy buy,people work great not imagine life will produc...,careful job life careful need learn seperate ...,product poeple enjoy buy. people work great no...,"['people', 'work', 'great', 'imagine', 'life',...",people work great imagine life product tool ne...,"['careful', 'job', 'life', 'careful', 'need', ...",careful job life careful need learn seperate ...,"['product', 'poeple', 'enjoy', 'buy']",product poeple enjoy buy
1,1,Apple,2008-02-14,4,4.0,5.0,3.0,4.0,3.0,v,...,different job different creative product work,big positive enjoy job apple innovative change...,suppose con hard balance work home life,different job different creative product work....,"['big', 'positive', 'enjoy', 'job', 'apple', '...",big positive enjoy job apple innovative change...,"['suppose', 'con', 'hard', 'balance', 'work', ...",suppose con hard balance work home life,"['different', 'job', 'different', 'creative', ...",different job different creative product work
2,2,Apple,2008-02-15,5,4.0,5.0,5.0,5.0,5.0,v,...,work apple hard job love,work place like apple opportunity change way w...,people apple live bleed edge not know future h...,work apple hard job love. work place like appl...,"['work', 'place', 'like', 'apple', 'opportunit...",work place like apple opportunity change way w...,"['people', 'apple', 'live', 'bleed', 'edge', '...",people apple live bleed edge know future hold ...,"['work', 'Apple', 'hard', 'job', 'love']",work Apple hard job love
3,3,Apple,2008-04-20,5,5.0,5.0,5.0,4.0,5.0,v,...,retail job altogether exciting thrilling life ...,despite fact boil sell product apple make feel...,intensive training mandatory meeting month pos...,retail job altogether exciting thrilling life ...,"['despite', 'fact', 'boil', 'sell', 'product',...",despite fact boil sell product apple feel like...,"['intensive', 'training', 'mandatory', 'meetin...",intensive training mandatory meeting month pos...,"['retail', 'job', 'altogether', 'exciting', 't...",retail job altogether exciting thrilling life ...
4,4,Apple,2008-04-20,2,3.0,5.0,2.0,3.0,1.0,x,...,bad choice,benefit fulltime employee great stock option r...,confidentiality not fart getting reprimand way...,bad choice. benefit fulltime employee great st...,"['benefit', 'fulltime', 'employee', 'great', '...",benefit fulltime employee great stock option r...,"['confidentiality', 'fart', 'get', 'reprimand'...",confidentiality fart get reprimand way feel li...,"['bad', 'choice']",bad choice


In [ ]:
# S’assurer que les colonnes sont bien des listes de mots

In [7]:
print(df[['proceeded_pros_clean', 'proceeded_cons_clean', 'proceeded_headline']].head())

                                proceeded_pros_clean  \
0  ['people', 'work', 'great', 'imagine', 'life',...   
1  ['big', 'positive', 'enjoy', 'job', 'apple', '...   
2  ['work', 'place', 'like', 'apple', 'opportunit...   
3  ['despite', 'fact', 'boil', 'sell', 'product',...   
4  ['benefit', 'fulltime', 'employee', 'great', '...   

                                proceeded_cons_clean  \
0  ['careful', 'job', 'life', 'careful', 'need', ...   
1  ['suppose', 'con', 'hard', 'balance', 'work', ...   
2  ['people', 'apple', 'live', 'bleed', 'edge', '...   
3  ['intensive', 'training', 'mandatory', 'meetin...   
4  ['confidentiality', 'fart', 'get', 'reprimand'...   

                                  proceeded_headline  
0              ['product', 'poeple', 'enjoy', 'buy']  
1  ['different', 'job', 'different', 'creative', ...  
2           ['work', 'Apple', 'hard', 'job', 'love']  
3  ['retail', 'job', 'altogether', 'exciting', 't...  
4                                  ['bad', 'choice'

In [ ]:
# Si cela n'avait pas été le cas, il aurait fallu les convertir en utilisant le code ci-dessous

In [8]:
import ast

for col in ['proceeded_pros_clean', 'proceeded_cons_clean', 'proceeded_headline']:
    df[col] = df[col].apply(ast.literal_eval)

In [ ]:
# Fusionner toutes les phrases dans une seule liste

In [9]:
sentences = []

for col in ['proceeded_pros_clean', 'proceeded_cons_clean', 'proceeded_headline']:
    sentences.extend(df[col].tolist())

In [ ]:
# 4. Entraîner le modèle Word2Vec
# Critères pour choisir la taille des vecteurs
# Corpus petit ou spécialisé : 
    # Privilégie des dimensions plus faibles (par exemple, 50 à 100). 
    # Cela limite le risque de surapprentissage et réduit le temps de calcul.

In [10]:
from gensim.models import Word2Vec

model = Word2Vec(
    sentences=sentences,
    vector_size=100,   # taille des vecteurs, à ajuster selon tes besoins
    window=5,          # fenêtre de contexte
    min_count=1,       # inclure tous les mots
    workers=4,         # nombre de threads
    sg=0               # 0 = CBOW, 1 = Skip-gram
)

In [11]:
vector = model.wv['salary']
print(vector)

[ 0.6231419   2.7783692   0.43573925  0.26664454  0.83616114  1.1635505
 -3.272921    0.40396753  1.3535467   1.5555717  -1.8092332  -2.1795092
  0.44882753 -2.271098    3.1106238   0.75815994 -1.998053    0.6097913
  0.21070103  1.1313021  -0.49298668 -1.4820007  -0.85251     0.09825359
 -0.1350426   0.02609222  0.913047    0.911184   -0.60629034 -0.03579746
 -1.7421273  -0.35969207  1.2662288  -0.90622604  0.3124099  -0.21543264
 -0.05556925 -0.42842287 -3.8405209   0.82354987 -0.67756027  1.2150216
  0.8805334  -1.3767258   0.9524264  -0.23537594  1.0207715  -2.0161662
 -2.235936    0.6174643  -0.28007764 -1.5416212  -1.5773771   0.492383
 -2.161738    2.3891754  -0.46700814 -2.5558128   1.007365    2.4999862
 -1.1862602   2.4464507   0.8200709   1.2181935  -2.6370876   1.0471522
 -0.56123626 -1.2093345   1.8548585   0.7626123  -0.5710631  -1.2679613
 -0.60822034  0.22916527 -1.6973562   0.48557237  1.2573905   0.04712638
 -0.90680885 -0.8259504   0.33334932  3.1580582   0.7204288  

In [12]:
print(model.wv.most_similar('salary'))

[('pay', 0.8810529708862305), ('compensation', 0.8360010981559753), ('payment', 0.7016961574554443), ('wage', 0.6660373210906982), ('remuneration', 0.643237829208374), ('comp', 0.6143624186515808), ('payout', 0.6057527661323547), ('package', 0.5947991013526917), ('bonus', 0.5689540505409241), ('income', 0.5533973574638367)]


In [13]:
vector = model.wv['promotion']
print(vector)

[ 0.06057146  1.4585768   1.0980402  -1.2803814   1.416492    2.2287734
 -2.356007   -0.4855688  -0.31060767 -0.23813938 -2.6076844  -2.3543088
  0.8744324  -0.8321924   2.6591284   1.3680556  -0.8759006  -0.82134914
  1.2289006   0.71224993  0.2785743  -1.3427585  -0.7493499   0.1606556
 -0.7646058  -0.56698036 -2.8744676  -1.6430894   0.08546175  1.6012802
 -0.55178094  0.28128183 -0.66556686  0.09639806 -1.1565306   0.97237676
  1.6318731  -1.1143591   1.1913486   0.3290324   0.93606585  0.23990902
  0.4662049  -1.5740404   2.8754108  -0.35483593  0.8901316  -1.8076609
 -2.6244977   2.3272407   1.6866957   1.5139852  -1.7533467  -1.3514978
 -0.55206704  0.81267077  0.02424083 -0.23128171 -0.08063545  2.8200293
 -2.8661137  -0.07102188 -0.47155675  2.467098   -0.81848603  0.934428
  1.7494601  -2.3843665   3.2111719   1.2899367  -1.5774255  -0.37247762
 -0.36367652  0.8008148  -3.7985945   1.3309115   0.25139067 -3.4482546
 -1.2379895   1.2657844  -0.23226178 -1.0476334   0.08810489 

In [14]:
print(model.wv.most_similar('promotion'))

[('promote', 0.6778565645217896), ('merit', 0.6278956532478333), ('progression', 0.6202023029327393), ('promo', 0.6029311418533325), ('advancement', 0.592108428478241), ('appraisal', 0.5538952350616455), ('evaluation', 0.550998866558075), ('bonus', 0.5478418469429016), ('promotional', 0.5404813885688782), ('seniority', 0.5301493406295776)]


In [16]:
print(model.wv.most_similar('openspace'))

[('unclean', 0.7230355739593506), ('overcrowd', 0.7227228283882141), ('spacious', 0.7025654315948486), ('humid', 0.6940626502037048), ('overcrowded', 0.690753161907196), ('hospitable', 0.6901567578315735), ('yucky', 0.6898556351661682), ('roach', 0.6878694891929626), ('conditioner', 0.6855373978614807), ('decorate', 0.683878481388092)]


In [17]:
print(model.wv.most_similar('openspace'))

[('unclean', 0.7230355739593506), ('overcrowd', 0.7227228283882141), ('spacious', 0.7025654315948486), ('humid', 0.6940626502037048), ('overcrowded', 0.690753161907196), ('hospitable', 0.6901567578315735), ('yucky', 0.6898556351661682), ('roach', 0.6878694891929626), ('conditioner', 0.6855373978614807), ('decorate', 0.683878481388092)]


In [18]:
model.save("word2vec_pros.model")

In [ ]:
# Créer la colonne cible sentiment

In [19]:
df['sentiment'] = df['overall_rating'].apply(lambda x: 1 if x >= 3 else 0)

In [ ]:
# Charger le modèle Word2Vec

In [20]:
model = Word2Vec.load("word2vec_pros.model")

In [ ]:
# Transformer chaque colonne en vecteur

In [21]:
import numpy as np
def get_sentence_vector(tokens, model):
    vectors = [model.wv[word] for word in tokens if word in model.wv]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

df['vec_pros'] = df['proceeded_pros_clean'].apply(lambda x: get_sentence_vector(x, model))
df['vec_cons'] = df['proceeded_cons_clean'].apply(lambda x: get_sentence_vector(x, model))
df['vec_headline'] = df['proceeded_headline'].apply(lambda x: get_sentence_vector(x, model))

In [ ]:
# Concaténer les vecteurs des trois colonnes

In [22]:
df['vector'] = df.apply(
    lambda row: np.concatenate([row['vec_pros'], row['vec_cons'], row['vec_headline']]),
    axis=1
)

In [ ]:
# Préparer les données pour la régression logistique = créer X (features) et y (cible)

In [23]:
X = np.vstack(df['vector'].values)
y = df['sentiment'].values

In [ ]:
# Diviser en train/test

In [24]:
!pip install scikit-learn

In [25]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Standardiser les données

In [26]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# entrainer la regression logistique

In [27]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train_scaled, y_train)

LogisticRegression(max_iter=1000)

In [ ]:
# Évaluer le modèle

In [28]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = clf.predict(X_test_scaled)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification report:\n", classification_report(y_test, y_pred))

Accuracy: 0.8769621682665161
Classification report:
               precision    recall  f1-score   support

           0       0.64      0.27      0.38      9861
           1       0.89      0.98      0.93     60979

    accuracy                           0.88     70840
   macro avg       0.77      0.62      0.65     70840
weighted avg       0.86      0.88      0.85     70840



In [ ]:
# Accuracy (précision globale)
    # 0.88 (88%)
    # Cela signifie que le modèle prédit correctement la classe (positif ou négatif) pour 88% des avis du test.

In [ ]:
# Classe 0 : négatif (overall_rating < 3)
    # precision 0.64 : Quand le modèle prédit "négatif", il a raison 64% du temps.
    # recall 0.27 : Il ne détecte que 27% des vrais négatifs (beaucoup de négatifs sont ratés).
    # f1-score 0.38 : Score global faible pour cette classe.

# Classe 1 : positif (overall_rating ≥ 3)
    # precision 0.89 : Quand il prédit "positif", il a raison 89% du temps.
    # recall 0.98 : Il détecte presque tous les vrais positifs.
    # f1-score 0.93 : Excellent score global pour cette classe.

In [ ]:
# Pourquoi ce déséquilibre ?
    # Support : Beaucoup plus d’avis positifs (60979) que négatifs (9861).
    # Ton modèle "apprend" surtout à reconnaître les positifs, et a du mal à repérer les négatifs 
    # (problème classique en cas de classes déséquilibrées).

In [ ]:
# Résumé
    # Ton modèle détecte très bien les avis positifs, mais rate beaucoup de négatifs.
    # C’est dû au déséquilibre des classes.
    # Pour corriger, essaie d’abord class_weight='balanced' dans ton modèle.


# Ce que fait class_weight='balanced'
# Cela donne plus de poids aux exemples de la classe minoritaire (ici, les avis négatifs).
# Le modèle va donc essayer de mieux détecter les avis négatifs, même s’ils sont moins nombreux.

In [ ]:
# Crée un nouveau modèle avec class_weight='balanced'

In [29]:
clf_balanced = LogisticRegression(max_iter=1000, class_weight='balanced')

In [ ]:
# Entraîne-le sur les mêmes données

In [30]:
clf_balanced.fit(X_train_scaled, y_train)

LogisticRegression(class_weight='balanced', max_iter=1000)

In [ ]:
# Prédit sur le test set

In [31]:
y_pred_balanced = clf_balanced.predict(X_test_scaled)

In [ ]:
# Affiche les nouveaux résultats

In [32]:
print("Accuracy (balanced):", accuracy_score(y_test, y_pred_balanced))
print("Classification report (balanced):\n", classification_report(y_test, y_pred_balanced))

Accuracy (balanced): 0.7617306606437041
Classification report (balanced):
               precision    recall  f1-score   support

           0       0.34      0.78      0.48      9861
           1       0.95      0.76      0.85     60979

    accuracy                           0.76     70840
   macro avg       0.65      0.77      0.66     70840
weighted avg       0.87      0.76      0.79     70840



In [ ]:
# Classe 0 (négatif) :
    # Precision 0.34 : Quand le modèle prédit "négatif", il a raison 34% du temps.
    # Recall 0.78 : Il détecte 78% des vrais négatifs (grosse amélioration par rapport à avant).
    # f1-score 0.48 : Score global bien meilleur sur cette classe.

# Classe 1 (positif) :
    # Precision 0.95 : Quand il prédit "positif", il a raison 95% du temps.
    # Recall 0.76 : Il détecte 76% des vrais positifs (c’est moins qu’avant, mais plus équilibré).

In [ ]:
# Ce que ça veut dire
    # Avant, ton modèle "oubliait" presque tous les négatifs (classe 0), mais était très bon sur les positifs.
    # Maintenant, ton modèle détecte beaucoup mieux les négatifs, même si la précision globale baisse un peu.
    # C’est normal : quand on rééquilibre, on sacrifie un peu la performance sur la classe majoritaire pour mieux traiter la classe minoritaire.

In [ ]:
# Sinon, tu peux déjà utiliser ce modèle pour détecter les avis négatifs avec bien plus de fiabilité qu’avant.

# Résumé
    # Ton modèle détecte maintenant beaucoup mieux les avis négatifs.
    # C’est le compromis classique quand on rééquilibre des classes déséquilibrées.
    # Tu peux continuer à améliorer, ou utiliser ce modèle selon tes besoins.